In [1]:
import utils.helpers as helpers
from utils.paths import *


import SimpleITK  # conda install -c https://conda.anaconda.org/simpleitk SimpleITK
import numpy
import pandas
import numpy as np
import pandas as pd
import ntpath
import cv2  # conda install -c https://conda.anaconda.org/menpo opencv3
import shutil
import random
import math
import multiprocessing
import os
import glob
import h5py
from joblib import Parallel, delayed
from tqdm import tqdm
import matplotlib.pyplot as plt
import skimage
import sklearn
import time

from typing import List, Tuple
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from skimage.transform import resize
from sklearn.utils import shuffle

from keras.optimizers import Adam, SGD
from keras.layers import Input, Convolution2D, MaxPooling2D, UpSampling2D, merge, Dropout, BatchNormalization,SpatialDropout2D,Convolution3D,MaxPooling3D, UpSampling3D, Flatten, Dense
from keras.models import Model, load_model, model_from_json
from keras.metrics import binary_accuracy, binary_crossentropy, mean_squared_error, mean_absolute_error
from keras import backend as K
from keras.callbacks import ModelCheckpoint, Callback, LearningRateScheduler,EarlyStopping
from keras.backend.tensorflow_backend import set_session
from keras.utils.vis_utils import plot_model
from keras.initializers import glorot_normal, he_uniform 

random.seed(1321)
numpy.random.seed(1321)

import warnings
warnings.filterwarnings("ignore")

K.set_image_dim_ordering('th') 

Using TensorFlow backend.


## 训练模型

In [2]:
src = PATH['model_train']
model_paths = PATH['model_paths']

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_coef_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return ((2. * intersection + 1) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1))


def unet_model(dropout_rate,learn_rate, width):
    inputs = Input((1, 512, 512))
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="relu")(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(pool1)
    conv2 = BatchNormalization(axis = 1)(conv2)
    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(pool2)
    conv3 = BatchNormalization(axis = 1)(conv3)
    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(pool3)
    conv4 = BatchNormalization(axis = 1)(conv4)
    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="relu")(pool4)
    conv5 = BatchNormalization(axis = 1)(conv5)
    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="relu")(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = SpatialDropout2D(dropout_rate)(up6)
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv6)
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = SpatialDropout2D(dropout_rate)(up7)
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv7)
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = SpatialDropout2D(dropout_rate)(up8)
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv8)
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = SpatialDropout2D(dropout_rate)(up9)
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv9)
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv9)
    conv10 = Convolution2D(1, (1, 1), activation="sigmoid")(conv9)

    model = Model(input=inputs, output=conv10)
    #model.summary()
    #model.compile(optimizer=Adam(lr=learn_rate), loss=dice_coef_loss, metrics=[dice_coef])
    model.compile(optimizer=SGD(lr=learn_rate, momentum=0.9, nesterov=True), loss=dice_coef_loss, metrics=[dice_coef])
    
    #plot_model(model, to_file='model1.png',show_shapes=True)
    return model

In [3]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_coef_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return ((2. * intersection + 1) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1))


def unet_model(dropout_rate,learn_rate, width):
    inputs = Input((1, 512, 512))
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="relu")(inputs)
    conv1 = SpatialDropout2D(dropout_rate)(conv1)
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(pool1)
    conv2 = SpatialDropout2D(dropout_rate)(conv2)
    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv2)
    conv2 = BatchNormalization(axis = 1)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(pool2)
    conv3 = SpatialDropout2D(dropout_rate)(conv3)
    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv3)
    conv3 = BatchNormalization(axis = 1)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(pool3)
    conv4 = SpatialDropout2D(dropout_rate)(conv4)
    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv4)
    conv4 = BatchNormalization(axis = 1)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="relu")(pool4)
    conv5 = SpatialDropout2D(dropout_rate)(conv5)
    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="relu")(conv5)
    conv5 = BatchNormalization(axis = 1)(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)    
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(up6)
    conv6 = SpatialDropout2D(dropout_rate)(conv6)
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv6)
    conv6 = BatchNormalization(axis = 1)(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)    
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(up7)
    conv7 = SpatialDropout2D(dropout_rate)(conv7)
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv7)
    conv7 = BatchNormalization(axis = 1)(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)    
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(up8)
    conv8 = SpatialDropout2D(dropout_rate)(conv8)
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv8)
    conv8 = BatchNormalization(axis = 1)(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)    
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="relu")(up9)
    conv9 = SpatialDropout2D(dropout_rate)(conv9)
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv9)
    conv9 = BatchNormalization(axis = 1)(conv9)
    conv10 = Convolution2D(1, (1, 1), activation="sigmoid")(conv9)

    model = Model(input=inputs, output=conv10)
    #model.summary()
    #model.compile(optimizer=Adam(lr=learn_rate), loss=dice_coef_loss, metrics=[dice_coef])
    model.compile(optimizer=SGD(lr=learn_rate, momentum=0.9, nesterov=True), loss=dice_coef_loss, metrics=[dice_coef])
    
    #plot_model(model, to_file='model1.png',show_shapes=True)
    return model

def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
	return -dice_coef(y_true, y_pred)

'''
The UNET model is compiled in this function.
'''
def unet_model():
	inputs = Input((1, 512, 512))
	conv1 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(inputs)
	conv1 = SpatialDropout2D(0.2)(conv1)
	conv1 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv1)
	pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

	conv2 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool1)
	conv2 = SpatialDropout2D(0.2)(conv2)
	conv2 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv2)
	pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

	conv3 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool2)
	conv3 = SpatialDropout2D(0.2)(conv3)
	conv3 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv3)
	pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

	conv4 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool3)
	conv4 = SpatialDropout2D(0.2)(conv4)
	conv4 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv4)
	pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

	conv5 = Convolution2D(1024, 3, 3, activation='relu', border_mode='same')(pool4)
	conv5 = SpatialDropout2D(0.2)(conv5)
	conv5 = Convolution2D(1024, 3, 3, activation='relu', border_mode='same')(conv5)

	up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
	conv6 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(up6)
	conv6 = SpatialDropout2D(0.2)(conv6)
	conv6 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv6)

	up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
	conv7 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up7)
	conv7 = Dropout(0.2)(conv7)
	conv7 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv7)

	up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
	conv8 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up8)
	conv8 = SpatialDropout2D(0.2)(conv8)
	conv8 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv8)

	up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
	conv9 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up9)
	conv9 = SpatialDropout2D(0.2)(conv9)
	conv9 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv9)

	conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

	model = Model(input=inputs, output=conv10)
	model.summary()
	model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

	return model

In [4]:
def unet_fit(name, check_name = None):
    data_gen_args = dict(rotation_range=90.,   
                     width_shift_range=0.3,  
                     height_shift_range=0.3,   
                     horizontal_flip=True,   
                     vertical_flip=True, 
                     )
    from keras.preprocessing.image import ImageDataGenerator
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    
    image_generator = image_datagen.flow_from_directory(
        src,
        class_mode=None,
        classes=['lung'],
        seed=seed,
        target_size=(512,512),
        color_mode="grayscale",
        batch_size=1)

    mask_generator = mask_datagen.flow_from_directory(
        src,
        class_mode=None,
        classes=['nodule'],
        seed=seed,
        target_size=(512,512),
        color_mode="grayscale",
        batch_size=1)

    # combine generators into one which yields image and masks
    train_generator = zip(image_generator, mask_generator)
        
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 50, 
                                   verbose = 1),
    ModelCheckpoint(model_paths + '{}.h5'.format(name), 
                        monitor='val_loss', 
                        verbose = 0, save_best_only = True)]
    if check_name is not None:
        check_model = model_paths + '{}.h5'.format(check_name)
        model = load_model(check_model, 
                           custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
    else:
        model = unet_model(dropout_rate = 0.2, learn_rate = 1e-3, width = 32)   
        #model = get_unet(1e-5)        
        
    model.fit_generator(
        train_generator,
        epochs=300,
        verbose =2, 
        callbacks = callbacks,
        steps_per_epoch=3700,
        validation_data = train_generator,
        nb_val_samples = 800)
    return

In [5]:
unet_fit('final_fenge_170629_2')
#unet_fit('final_fenge_170629_2','final_fenge_170629_2')

Found 3711 images belonging to 1 classes.
Found 3711 images belonging to 1 classes.
Epoch 1/300
511s - loss: -8.3740e-01 - dice_coef: 0.8374 - val_loss: -9.0781e-01 - val_dice_coef: 0.9078
Epoch 2/300
506s - loss: -1.1724e+00 - dice_coef: 1.1724 - val_loss: -1.0515e+00 - val_dice_coef: 1.0515
Epoch 3/300
507s - loss: -1.3064e+00 - dice_coef: 1.3064 - val_loss: -1.2745e+00 - val_dice_coef: 1.2745
Epoch 4/300
506s - loss: -1.3575e+00 - dice_coef: 1.3575 - val_loss: -1.3381e+00 - val_dice_coef: 1.3381
Epoch 5/300
505s - loss: -1.3848e+00 - dice_coef: 1.3848 - val_loss: -1.2807e+00 - val_dice_coef: 1.2807
Epoch 6/300
505s - loss: -1.4213e+00 - dice_coef: 1.4213 - val_loss: -1.3843e+00 - val_dice_coef: 1.3843
Epoch 7/300
504s - loss: -1.4263e+00 - dice_coef: 1.4263 - val_loss: -1.3606e+00 - val_dice_coef: 1.3606
Epoch 8/300
505s - loss: -1.4397e+00 - dice_coef: 1.4397 - val_loss: -3.2074e-01 - val_dice_coef: 0.3207
Epoch 9/300
507s - loss: -1.4404e+00 - dice_coef: 1.4404 - val_loss: -1.3525

Epoch 78/300
500s - loss: -1.6159e+00 - dice_coef: 1.6159 - val_loss: -1.4483e+00 - val_dice_coef: 1.4483
Epoch 79/300
503s - loss: -1.6426e+00 - dice_coef: 1.6426 - val_loss: -1.4188e+00 - val_dice_coef: 1.4188
Epoch 80/300
503s - loss: -1.6504e+00 - dice_coef: 1.6504 - val_loss: -1.4423e+00 - val_dice_coef: 1.4423
Epoch 81/300
503s - loss: -1.6466e+00 - dice_coef: 1.6466 - val_loss: -1.4213e+00 - val_dice_coef: 1.4213
Epoch 82/300
502s - loss: -1.6328e+00 - dice_coef: 1.6328 - val_loss: -1.4772e+00 - val_dice_coef: 1.4772
Epoch 83/300
503s - loss: -1.6590e+00 - dice_coef: 1.6590 - val_loss: -1.4466e+00 - val_dice_coef: 1.4466
Epoch 84/300
503s - loss: -1.6102e+00 - dice_coef: 1.6102 - val_loss: -1.3999e+00 - val_dice_coef: 1.3999
Epoch 85/300
502s - loss: -1.6389e+00 - dice_coef: 1.6389 - val_loss: -1.4277e+00 - val_dice_coef: 1.4277
Epoch 86/300
504s - loss: -1.6346e+00 - dice_coef: 1.6346 - val_loss: -1.5393e+00 - val_dice_coef: 1.5393
Epoch 87/300
503s - loss: -1.6378e+00 - dice_c